In [3]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import *

# Spark 101 

## Create a spark data frame that contains your favorite programming languages.

In [90]:
languages = {'languages': ['python', 'java', 'html', 'javascript', 'typescript', 'c#', 'c++', 'c', 'sql']}

### - The name of the column should be language

In [91]:
df = spark.createDataFrame(pd.DataFrame(languages))

In [92]:
df.show(5)

+----------+
| languages|
+----------+
|    python|
|      java|
|      html|
|javascript|
|typescript|
+----------+
only showing top 5 rows



### - View the schema of the dataframe

In [93]:
df.schema

StructType(List(StructField(languages,StringType,true)))

### - Output the shape of the dataframe

In [94]:
df.

AttributeError: 'DataFrame' object has no attribute 'shape'

### - Show the first 5 records in the dataframe